# BI Exam May 2025: COVID-19 Data

#### Created by Group 7 - Kamilla, Jeanette, Juvena

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
import sklearn.metrics as sm
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import explained_variance_score

# Set plot styles for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Data Preparation

### 1. Load the Data

Now that we have our tools ready, the next step is to load the COVID-19 dataset into Python so we can start analyzing it.

In this case, we’re working with a single dataset:

- **OWID COVID-19 Latest Data**: a CSV file that contains country-level information on cases, deaths, vaccinations, testing, and various socioeconomic indicators.

We'll use Pandas to read the CSV file and store it as a DataFrame. To make our code cleaner and reusable, we'll define a simple function that loads the data and performs some initial checks. This way, we can easily reload or replace the dataset if needed in future steps.

In [ ]:
# File paths for the covid datasets. (dataset: last updated 2024-08-04)
dataset_covid = 'Data/owid-covid-latest.csv'

# Function to load the Excel files
def load_csv_to_dataframe(file_path):
    # Reads the Excel file and skips the first row if it contains a description or title
    df = pd.read_csv(file_path)
    return df

# Load datasets
print("..Loading COVID-19 dataset")
df_covid = load_csv_to_dataframe(dataset_covid)

### 2. Explore the Data

After loading the dataset, we want to explore it to understand what kind of information it contains and how it's structured.

To do this, we can use several helpful Pandas functions such as `shape`, `types`, `info()`, `head()`, `tail()`, `sample()`, `describe()` and `isnull().sum()`. These functions will give us insights into the number of rows and columns, the data types of each column, a summary of the data, and any missing values. 

This exploration is crucial as it helps us identify potential issues or areas that need further cleaning or transformation before we proceed with our analysis. 

In [ ]:
# Check the shape of the DataFrame (rows, columns)
df_covid.shape

In [ ]:
# Display the types of attributes (colum names) in the DataFrame
df_covid.dtypes

In [ ]:
# Gives an overview of the DataFrame
df_covid.info()

In [ ]:
# Display the first 5 rows of the DataFrame
df_covid.head()

In [ ]:
# Display the last 5 rows of the DataFrame
df_covid.tail()

In [ ]:
# Display a random sample of 5 rows from the DataFrame
df_covid.sample(5)

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_covid.describe()

##### **2.1 Summary of exploring the data**

After exploring the dataframe, we found that it contains a large number of columns, many of which are not useful for our analysis or modeling goals. While some columns provide valuable information (like total cases, deaths, and vaccination rates), others are either redundant, mostly empty, or irrelevant.

This highlights the need for a thorough data cleaning step to remove unnecessary columns, handle missing values, and focus only on the most relevant features for our machine learning tasks.

### 3. Clean the Data

After loading and exploring the data, we need to clean it to ensure that our analysis is accurate and meaningful. Data cleaning involves several steps, including: checking for missing values, removing duplicates, and converting data types.

We start by doing a bit of cleaning of the big dataset, to remove rows and columns that are not relevant for our futher analysis and before we seperate the data into more specific datasets.

In [ ]:
# Check for missing values in the DataFrame
df_covid.isnull().sum()

The output above shows that many columns contain no values at all, so we will remove them to clean up the dataset.

In [ ]:
# Before cleaning the data, we want to remove irrelevant OWID aggregate rows—such as those representing high-income, low-income, and other income groupings.
rows_to_remove = ["OWID_UMC", "OWID_WRL", "OWID_LMC", "OWID_LIC", "OWID_HIC"]
df_removed_rows = df_covid[~df_covid["iso_code"].isin(rows_to_remove)]

We are removing the 'low-income countries', 'lower-middle-income countries', 'upper-middle-income countries', 'high-income countries' and 'world' categories because they are too broad and lack specific country-level detail, making it difficult to draw meaningful conclusions without relying on assumptions.

In [ ]:
# Checking if the above rows were removed
print(f"{df_covid.shape}")
print(f"Removed the {df_covid.shape[0] - df_removed_rows.shape[0]} OWID rows from the dataframe.")

In [ ]:
# We will drop all columns with no values at all like; excess_mortality_cumulative_absolute, excess_mortality_cumulative etc.
df_covid_removed_columns = df_removed_rows.dropna(axis=1, how='all')

In [ ]:
# Check whether the columns were removed
print(f"COVID dataframe shape after removing columns: {df_covid_removed_columns.shape}")
print(f"Removed {df_covid.shape[1] - df_covid_removed_columns.shape[1]} columns from the dataframe.")

#### 3.1 Separating the data into different datasets

Now we separate the age-level and health-level data into their own DataFrames so that we can clean and process them independently from the country-level data. This allows us to apply different cleaning steps based on the nature of the data, since data may have different structures or missing values compared to individual countries.

##### 3.1.1 Separating the age-level data

In [ ]:
# Function to filter the DataFrame based on a list of values
def filter_dataframe(df, values, filter_type='rows', row_filter_column=None):
    if filter_type == 'rows':
        if row_filter_column is None:
            raise ValueError("Must specify 'row_filter_column' when filtering rows.")
        return df[df[row_filter_column].isin(values)]
    elif filter_type == 'columns':
        # Keep only columns present in df and in values list (avoid key error)
        columns_to_keep = [col for col in values if col in df.columns]
        return df[columns_to_keep]
    else:
        raise ValueError("filter_type must be either 'rows' or 'columns'")

In [ ]:
# We are using the function above to seperate the age-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "median_age", "aged_65_older", "aged_70_older", "life_expectancy"]
df_age = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [ ]:
# Check if the rows were secured
df_age

We now have a new seperate dataframe called `df_age` that contains the age-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Check for missing values in the DataFrame
df_age.isnull().sum()

In [ ]:
# Drop all the rows with NaN values in the 'median_age' column
df_age_cleaned = df_age.dropna(subset=['median_age'])

In [ ]:
# Do another check for missing values in the DataFrame
df_age_cleaned.isnull().sum()

There are still some missing values in the `df_age_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# method for replacing cell with median 
def fill_na_with_median(df, column_name):
    median_value = df[column_name].median()
    print(f"Median of '{column_name}': {median_value:.2f}")
    df[column_name].fillna(median_value, inplace=True)

In [ ]:
# Fill NaN values with the median for the columns; total_deaths_per_million, aged_65_older and aged_70_older
fill_na_with_median(df_age_cleaned, "total_deaths_per_million")
fill_na_with_median(df_age_cleaned, "aged_65_older")
fill_na_with_median(df_age_cleaned, "aged_70_older")
df_age_cleaned

In [ ]:
# Check for duplicates in the DataFrame
df_age_cleaned.duplicated().sum()

##### 3.1.2 Separating the health-level data

In [ ]:
# We are using the same function as above to seperate the health-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "life_expectancy"]
df_health = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [ ]:
# Check if the rows were secured
df_health

We now have a new seperate dataframe called `df_health` that contains health-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Check for missing values in the DataFrame
df_health.isnull().sum()

In [ ]:
# Drop all the rows with NaN values in the 'female_smokers' column
df_health_cleaned = df_health.dropna(subset=['female_smokers'])

In [ ]:
# Do another check for missing values in the DataFrame
df_health_cleaned.isnull().sum()

There are still some missing values in the `df_health_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Fill NaN values with the median for the columns; cardiovasc_death_rate and male_smokers
fill_na_with_median(df_health_cleaned, "cardiovasc_death_rate")
fill_na_with_median(df_health_cleaned, "male_smokers")
df_health_cleaned

In [ ]:
# Check for duplicates in the DataFrame
df_health_cleaned.duplicated().sum()

#### 3.2 Futher cleaning of the country-level data 

We have selected a subset of columns that we consider relevant for our analysis. This subset includes columns that provide information on total cases, deaths and population. By focusing on these columns, we can simplify our analysis and make it easier to draw meaningful conclusions.

In [ ]:
# We make a new dataframe with the columns we want to keep for future analysis.
columns_we_want_to_keep = [
    "iso_code", "continent", "location", "total_cases", "total_deaths",
    "total_cases_per_million", "total_deaths_per_million",
    "life_expectancy", "population"]

# Removes all other columns
df_covid = df_covid_removed_columns[columns_we_want_to_keep]

In [ ]:
# Check if the columns were removed
df_covid.info()

We then load another dataset so we can add data about the Human Development Index (HDI) for each country. The HDI is a composite index of life expectancy, education, and per capita income indicators, which are used to rank countries into four tiers of human development. This additional information will help us better understand the relationship between COVID-19 and various socioeconomic factors.

In [ ]:
# We load the new dataset
hdi = pd.read_csv('Data/human-development-index.csv')

Because we can then add a column with the HDI data for 2021 matching the countries in the covid dataset, because we only need data from the last year.

In [ ]:
# Filter HDI for 2021 only
hdi_2021 = hdi[hdi['Year'] == 2021]

# Merge using 'location' from df_covid and 'Entity' from hdi
df_merged = df_covid.merge(
    hdi_2021[['Entity', 'Human Development Index']], 
    left_on='location', 
    right_on='Entity', 
    how='left'
)

# Drop the extra 'Entity' column after merge, since we don't need it
df_merged = df_merged.drop(columns=['Entity'])

# Rename the column in the merged dataframe
df_merged = df_merged.rename(columns={'Human Development Index': 'human_development_index'})

In [ ]:
# Check how the dataset look and how we should proceed
df_merged

In [ ]:
# Shape of the dataframe after some cleaning
print(f"COVID dataframe shape after removing both some columns and rows: {df_merged.shape}")

We are isolating the remaining rows in the df_covid DataFrame to ensure it contains only country-level data. This allows us to clean the dataset and retain only the features that are most relevant for our analysis.

In [ ]:
# Since we seperated the OWID continent fields into it's own dataframe earlier, we now have to remove them again for the df_covid dataframe.
rows_to_remove = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_covid_removed_rows = df_merged[~df_merged['iso_code'].isin(rows_to_remove)]
df_covid_cleaned = df_covid_removed_rows.dropna(subset=['iso_code'])
df_covid_cleaned = df_covid_cleaned.drop(columns=['iso_code'])
df_covid_cleaned        

In [ ]:
# Check whether the rows were removed
print(f"COVID dataframe shape after removing rows: {df_covid_cleaned.shape}")
print(f"Removed {df_merged.shape[0] - df_covid_cleaned.shape[0]} rows from the dataframe.")
print(f"Removed {df_merged.shape[1] - df_covid_cleaned.shape[1]} column from the dataframe.")

In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

We can see a lot of missing values for the human_development_index column, so we will impute them with the HDI from the sovereign countries they belong too. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

In [ ]:
territory_to_country = {
    'American Samoa': 'United States',
    'Anguilla': 'United Kingdom',
    'Aruba': 'Netherlands',
    'Bermuda': 'United Kingdom',
    'Bonaire Sint Eustatius and Saba': 'Netherlands',
    'British Virgin Islands': 'United Kingdom',
    'Cayman Islands': 'United Kingdom',
    'Cook Islands': 'New Zealand',
    'Curacao': 'Netherlands',
    'Falkland Islands': 'United Kingdom',
    'Faroe Islands': 'Denmark',
    'French Guiana': 'France',
    'French Polynesia': 'France',
    'Gibraltar': 'United Kingdom',
    'Greenland': 'Denmark',
    'Guadeloupe': 'France',
    'Guam': 'United States',
    'Guernsey': 'United Kingdom',
    'Isle of Man': 'United Kingdom',
    'Jersey': 'United Kingdom',
    'Kosovo': 'Serbia',  # or leave as is if Kosovo has its own HDI
    'Martinique': 'France',
    'Mayotte': 'France',
    'Monaco': 'France',
    'Montserrat': 'United Kingdom',
    'Nauru': 'Nauru',
    'New Caledonia': 'France',
    'Niue': 'New Zealand',
    'North Korea': 'North Korea',
    'Northern Mariana Islands': 'United States',
    'Pitcairn': 'United Kingdom',
    'Puerto Rico': 'United States',
    'Reunion': 'France',
    'Saint Barthelemy': 'France',
    'Saint Helena': 'United Kingdom',
    'Saint Martin (French part)': 'France',
    'Saint Pierre and Miquelon': 'France',
    'Sint Maarten (Dutch part)': 'Netherlands',
    'Somalia': 'Somalia',
    'Tokelau': 'New Zealand',
    'Turks and Caicos Islands': 'United Kingdom',
    'United States Virgin Islands': 'United States',
    'Vatican': 'Italy',
    'Wallis and Futuna': 'France'
}

In [ ]:
# Map the territory to its sovereign country:
df_covid_cleaned['hdi_source_country'] = df_covid_cleaned['location'].map(territory_to_country)

In [ ]:
# Create a lookup for HDI values of sovereign countries
hdi_lookup = df_covid_cleaned.set_index('location')['human_development_index'].to_dict()

In [ ]:
# Fill missing HDI values with the sovereign country’s HDI
df_covid_cleaned['human_development_index'] = df_covid_cleaned.apply(
    lambda row: hdi_lookup.get(row['hdi_source_country'], row['human_development_index']) 
    if pd.isna(row['human_development_index']) else row['human_development_index'],
    axis=1
)

df_covid_cleaned.drop(columns=['hdi_source_country'], inplace=True)

In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

In [ ]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

We have found data for the human_development_index for 2021 for Nauru and Somalia on the https://hdr.undp.org/ website. We will use this data to fill in the missing values for these two countries in our dataset. We weren't able to find data on the human_development_index for North Korea, so we will impute it with the median value of the HDI for the other countries in the dataset. 

We will also impute the missing values for the columns total_cases, total_deaths, total_cases_per_million, total_deaths_per_million and life_expectancy with the median value.

In [ ]:
# method for replacing cell with a value
def replace_cell(df, row_filter, column, value):
    df.loc[row_filter, column] = value

In [ ]:
# Replace missing HDI values for Nauru and Somalia and impute North Korea with median value
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Nauru', 'human_development_index', 0.692)
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Somalia', 'human_development_index', 0.385)
fill_na_with_median(df_covid_cleaned, "human_development_index")
fill_na_with_median(df_covid_cleaned, "total_cases")
fill_na_with_median(df_covid_cleaned, "total_deaths")
fill_na_with_median(df_covid_cleaned, "total_cases_per_million")
fill_na_with_median(df_covid_cleaned, "total_deaths_per_million")
fill_na_with_median(df_covid_cleaned, "life_expectancy")
df_covid_cleaned


In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

In [ ]:
# Check for duplicates in the DataFrame
df_covid_cleaned.duplicated().sum()

### 4. Hypotese 1: Higher population size is associated with higher total COVID-19 deaths, but not necessarily with higher deaths per capita. 


 #### 4.1 Explore

 #### 4.2 Data Modelling

### 5. Hypotese 2: Countries with a higher Human Development Index (HDI) have experienced lower COVID-19 death rates per capita

We chose to investigate this hypothesis because HDI reflects key aspects of a country’s development, such as healthcare quality, education, and living standards.
It seems reasonable to assume that countries with higher HDI might be better equipped to handle a health crisis like COVID-19, potentially resulting in lower death rates.

 #### 5.1 Explore

In [ ]:
df_covid_cleaned

In [ ]:
df_covid_cleaned.info()

In [ ]:
df_covid_cleaned.describe()


Now that we explored the new cleaned dataframe a bit, we can see that the df_covid_cleaned dataframe contains a more manageable number of columns and rows vs the original dataframe. The columns we have retained are relevant for our analysis, and we have removed unnecessary or redundant features.

##### 5.1.1 Check for outliers in the df_covid_cleaned

The next step in exploring the data is checking for outlier values that are unusually high or low compared to the rest of the data.

We use the IQR (Interquartile Range) method, which is a common way to detect outliers:

-  First, we calculate the first quartile (Q1) and third quartile (Q3) for each selected column.
- The IQR is the difference between Q3 and Q1.
- Any value that falls below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR is considered an outlier.

We apply this method to the two important features regarding our hypotheses: total_deaths_per_million and human_development_index. This helps us find any unusual data points that could affect the results of our analysis.

In [ ]:
# Check for outliers in the df_covid_cleaned dataframe using IQR method
print("\n..Checking for outliers in the df_covid_cleaned dataframe:")

# Loop through selected columns
for column in ['total_deaths_per_million', 'human_development_index']:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_covid_cleaned[column].quantile(0.25)
    Q3 = df_covid_cleaned[column].quantile(0.75)
    IQR = Q3 - Q1  # Interquartile Range

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find rows where the value is outside the normal range
    outliers = df_covid_cleaned[
        (df_covid_cleaned[column] < lower_bound) | 
        (df_covid_cleaned[column] > upper_bound)
    ]

    # Print the number of outliers found for the column
    print(f"  {column}: {len(outliers)} outliers detected")
    print(outliers[['location', column]])

##### 5.1.2 Conclusion of outliers: 
 There are detected 7 outliers in feature 'total_deaths_per_million' with countries there has very high death toll pr million compared to the other countries. It can have an effect on average and visualizations. 

 These outliers are likely not errors but reflect extreme yet valid data points related to the real impact of COVID-19 in certain countries. For this reason, we’ve chosen to keep them. Its possible that these values could provide valuable insights into how HDI may have had an impact on death rates per capita. Removing them might hide important patterns in the data.





##### 5.1.3 Visualize the impact of HDI on Covid-19 death rate

##### 5.1.3.1 Scatterplot

To explore whether a relationship exists between Human Development Index (HDI) and COVID-19 death rates per million, we use a scatterplot to visualize the distribution and potential correlation between the two variables.

In [ ]:
sns.scatterplot(data=df_covid_cleaned, x='human_development_index', y='total_deaths_per_million')
plt.title('Human Development Index vs Total Deaths per Million')
plt.show()

The above scatterplot shows no clear negative correlation between HDI and COVID-19 death rates. High HDI countries vary widely in death rates, suggesting that HDI alone does not explain the differences. Other factors likely play a role.

Countries with low HDI values do not consistently show higher death rates either, reinforcing that HDI alone is not a strong predictor of COVID-19 mortality. 

##### 5.1.3.2 Correlation matrix

In [ ]:
corr_matrix = df_covid_cleaned[['human_development_index', 'total_deaths_per_million']].corr()

In [ ]:
sns.heatmap(corr_matrix, annot=True)

The correlation matrix above shows a moderate positive correlation (0.47) between Human Development Index and COVID-19 death rates per million. This is surprising, as our hypothesis expected a negative correlation — that higher HDI would be linked to lower death rates. The result suggests that, in this dataset, countries with higher HDI tend to report higher death rates per million. This indicates that HDI alone does not explain the differences, and other factors likely influence the outcomes.

 #### 5.2 Data Modelling

##### 5.2.1 Linear regression (Supervised Machine Learning)

To further investigate the relationship between Human Development Index (HDI) and COVID-19 death rates per million, we apply linear regression. This method helps assess the strength and direction of the relationship between these two variables and allows us to evaluate whether HDI can be used to predict COVID-19 mortality rates across countries.

In [ ]:
# Choose dependent and independent variables

# independent
X = df_covid_cleaned[['human_development_index']]

# dependent
y = df_covid_cleaned[['total_deaths_per_million']]

In [ ]:
# Splitting the dataset into training and testing sets

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.20)

In [ ]:
# the shape of the subsets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Creating an instance of Linear Regression model
myreg = LinearRegression()

In [ ]:
# Fit it to our data
myreg.fit(X_train, y_train)
myreg

In [ ]:
# Get the calculated coefficients
a = myreg.coef_
b = myreg.intercept_

a

In [ ]:
b

In [ ]:
print(f"The model is a line, y = a * x + b, or y = {a} * x + {b}")

In [ ]:
y_predicted = myreg.predict(X_test)

In [ ]:
# Visualise the Linear Regression 
plt.title('Linear Regression: HDI vs COVID-19 Deaths per Million')
plt.scatter(X, y, color='green')
plt.plot(X_train, a*X_train + b, color='blue')
plt.plot(X_test, y_predicted, color='orange')
plt.xlabel('HDI')
plt.ylabel('Deaths per Million')
plt.show()

The above graph visualizes the relationship between HDI and COVID-19 deaths per million using a linear regression model. Each green dot represents a country. The orange line shows the model’s predicted trend based on the data. While there appears to be a slight upward trend, the data points are spread out, especially at higher HDI values, suggesting that the relationship might not be very strong.

In [ ]:
# Predict deaths pr million from HDI
hdi_value = 0.85
prediction= myreg.predict([[hdi_value]])
print(f"Predicted death rate for HDI {hdi_value}:", prediction)

In [ ]:
manual_prediction = a * hdi_value + b
print("Manual prediction:", manual_prediction)

In [ ]:
# Mean Absolute Error (MAE) is the mean of the absolute value of the errors
print("MAE:", metrics.mean_absolute_error(y_test, y_predicted))

# Mean Squared Error (MSE) is the mean of the squared errors
print("MSE:", mean_squared_error(y_test, y_predicted))

# Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_predicted)))

# R-squared: the proportion of the variation in the dependent variable that is predictable from the independent variable(s)
print("R² score:", r2_score(y_test, y_predicted))

##### 5.2.1.1 Conclusion of linear regression
Based on the results, the linear regression model dos not perform well.
The average error (MAE) is 774 and the root mean square error (RMSE) is over 1000, which means the predictions are far from the actual values.
The R² score is only 0.28, meaning that HDI explains just 28% of the differences in death rates between countries.
This suggests that HDI alone is not a good predictor of COVID-19 mortality, and that other factors likely play a more important role.

 #### 5.3 Additional Analysis - Nordic Comparison: HDI and Death Rates

Since our earlier results showed that HDI alone does not explain differences in COVID-19 death rates, we chose to examine the Nordic countries. These countries have very similar HDI levels and welfare systems, which makes them ideal for a focused comparison. This analysis helps test whether HDI has a consistent effect within a more uniform group and can either support or weaken our hypothesis.

In [ ]:
# Filter the DataFrame for Nordic countries
nordic_countries = ['Denmark', 'Sweden', 'Norway', 'Finland', 'Iceland']
df_nordic = df_covid_cleaned[df_covid_cleaned['location'].isin(nordic_countries)]


In [ ]:
# Select relevant columns 
df_nordic_subset = df_nordic[['location', 'human_development_index', 'total_deaths_per_million']]
df_nordic_subset

##### 5.3.1 Bar chart

In [ ]:
# Visualize the death rates using a bar chart

plt.figure(figsize=(8, 5))
sns.barplot(data=df_nordic_subset, x='location', y='total_deaths_per_million')
plt.title('COVID-19 Deaths per Million – Nordic Countries')
plt.xlabel('Country')
plt.ylabel('Deaths per Million')
plt.show()

##### 5.3.2 Conclusion of Nordic comparison

Despite similar HDI levels among the Nordic countries, there is a clear variation in COVID-19 death rates per million. Sweden shows the highest rate, while Iceland has the lowest. This indicates that even within a region with high and comparable development, other factors beyond HDI may strongly influence COVID-19 mortality.

 #### 5.4 Conclusion of Hypothesis 2 

The analysis does not support the hypothesis that countries with a higher Human Development Index (HDI) have experienced lower COVID-19 death rates per capita. Although HDI was expected to be a strong predictor, the results show only a weak to moderate positive correlation (0.47), and the linear regression model performed poorly (R² = 0.28). Additionally, the Nordic comparison showed large differences in death rates despite very similar HDI values. This suggests that HDI alone is not sufficient to explain COVID-19 mortality differences, and that other factors likely play a more significant role.

Although HDI reflects general development such as healthcare, education, and living standards, it may not capture specific pandemic-related factors like healthcare system capacity or testing infrastructure. Therefore, HDI alone may not be sufficient to explain variations in COVID-19 death rates, and other, more direct factors likely play a greater role.

### 6. Hypotese 3: Countries with a higher life expectancy and older populations (e.g. higher median age, % aged 65+, etc.) have experienced higher COVID-19 death rates

 #### 6.1 Explore

 #### 6.2 Data Modelling

### 7. Hypotese 4: Countries with higher prevalence of chronic health conditions (e.g. cardiovascular death rate, diabetes, smoking) have higher COVID-19 death rates

We chose to investigate this hypothesis because chronic conditions like heart disease, diabetes, and smoking are known to increase the risk of severe COVID-19 outcomes. Our goal is to examine if countries with higher rates of these conditions also experienced higher COVID-19 death rates.








 #### 7.1 Explore

In [ ]:
df_health_cleaned

In [ ]:
df_health_cleaned.info()

In [ ]:
df_health_cleaned.describe()

Now that we explored the new cleaned dataframe a bit, we can see that the df_health_cleaned dataframe contains a more manageable number of columns and rows vs the original dataframe. The columns we have retained are relevant for our analysis, and we have removed unnecessary or redundant features.

##### 7.1.1 Check for outliers in the df_health_cleaned

The next step in exploring the data is checking for outlier values that are unusually high or low compared to the rest of the data.
We use the IQR (Interquartile Range) method, which is a common way to detect outliers.

We apply this method to the five important features regarding our hypotheses: 
- cardiovasc_death_rate
- diabetes_prevalence
- female_smokers
- male_smokers
- total_deaths_per_million

This helps us find any unusual data points that could affect the results of our analysis.

In [ ]:
# Check for outliers in the df_health_cleaned dataframe using IQR method
print("\n..Checking for outliers in the df_health_cleaned dataframe:")

# Loop through selected columns
for column in ['cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 'total_deaths_per_million']:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_health_cleaned[column].quantile(0.25)
    Q3 = df_health_cleaned[column].quantile(0.75)
    IQR = Q3 - Q1  # Interquartile Range

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find rows where the value is outside the normal range
    outliers = df_health_cleaned[
        (df_health_cleaned[column] < lower_bound) | 
        (df_health_cleaned[column] > upper_bound)
    ]

    # Print the number of outliers found for the column
    print(f"  {column}: {len(outliers)} outliers detected")
    print(outliers[['location', column]])

##### 7.1.2 Conclusion of outliers: 
We identified several outliers, especially in diabetes_prevalence (7 outliers). Although these values can affect averages and visualizations, we chose to keep them because they likely reflect real-world differences between countries. Removing them could hide important patterns in how chronic health conditions relate to COVID-19 death rates.

##### 7.1.3 Visualize the correlation between health conditions and Covid-19 death rate

##### 7.1.3.1 Scatterplot

We use a scatterplot to visualize how each health condition is associated with COVID-19 death rates across countries. It helps reveal any visible patterns or trends in the data.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

sns.scatterplot(data=df_health_cleaned, x='cardiovasc_death_rate', y='total_deaths_per_million', ax=axes[0, 0])
axes[0, 0].set_title('Cardiovascular Death Rate vs COVID-19 Deaths per Million')

sns.scatterplot(data=df_health_cleaned, x='diabetes_prevalence', y='total_deaths_per_million', ax=axes[0, 1])
axes[0, 1].set_title('Diabetes Prevalence vs COVID-19 Deaths per Million')

sns.scatterplot(data=df_health_cleaned, x='female_smokers', y='total_deaths_per_million', ax=axes[1, 0])
axes[1, 0].set_title('Female Smokers vs COVID-19 Deaths per Million')

sns.scatterplot(data=df_health_cleaned, x='male_smokers', y='total_deaths_per_million', ax=axes[1, 1])
axes[1, 1].set_title('Male Smokers vs COVID-19 Deaths per Million')

plt.tight_layout()
plt.show()

The above scatterplots suggest only weak or no clear correlations between chronic health conditions and COVID-19 death rates across countries. Cardiovascular death rate and diabetes prevalence show no strong trend, as death rates remain spread across different values. Female smokers appear to have a slightly positive association with higher death rates, whereas male smokers show no clear pattern. Overall, the visualizations indicate that these individual health indicators alone may not strongly explain variations in COVID-19 mortality, and other factors likely contribute more significantly.

##### 7.1.3.2 Correlation matrix

We use a correlation matrix to examine the strength and direction of the relationships between the multiple health-related variables and COVID-19 death rates. It helps us compare all variables at once and identify potential patterns or associations

In [ ]:
corr_matrix = df_health_cleaned[['total_deaths_per_million', 'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers']].corr()

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap='YlOrRd')

The above matrix shows that female_smokers has the strongest positive correlation with COVID-19 death rates (0.58), while male_smokers shows a weak correlation (0.13), and cardiovascular_death_rate and diabetes_prevalence are slightly negative (-0.14 and -0.065). This suggests that none of the health conditions show a strong linear relationship with COVID-19 deaths, though female smoking stands out with a moderate correlation worth noting.

<!-- The above scatterplot shows no clear negative correlation between HDI and COVID-19 death rates. High HDI countries vary widely in death rates, suggesting that HDI alone does not explain the differences. Other factors likely play a role.

Countries with low HDI values do not consistently show higher death rates either, reinforcing that HDI alone is not a strong predictor of COVID-19 mortality -->

 #### 7.2 Data Modelling

##### 7.2.1 Multiple Linear regression (Supervised Machine Learning)

To further investigate the relationship between various health factors and COVID-19 death rates per million, we apply multiple linear regression. This approach helps us understand how strongly each factor contributes to differences in mortality rates, and whether these factors can collectively explain the variation in COVID-19 deaths across countries.

In [ ]:
# Create a list of the features names
feature_cols = ['cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers']

# Select only the relevant predictor variables (independent) from the dataframe
X = df_health_cleaned[feature_cols]

X.head()

In [ ]:
# Select the target variable (dependent variable) for prediction
y = df_health_cleaned['total_deaths_per_million']

# Print the first 5 values
y.head()

In [ ]:
# Check the type and shape of X
print(type(X))
print(X.shape)

In [ ]:
# Check the type and shape of y
print(type(y))
print(y.shape)

In [ ]:
# Splitting X and y into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.20)

In [ ]:
# The shape of the subsets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Creating an instance of Linear Regression model
linreg = LinearRegression()

# Fit the model to our training data
linreg.fit(X_train, y_train)
linreg

In [ ]:
# The intercept and coefficients are stored in system variables
print('b0 =', linreg.intercept_)
print('bi =', linreg.coef_)

In [ ]:
# Pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

In [ ]:
# Make predictions on the testing set
y_predicted = linreg.predict(X_test)
y_predicted

In [ ]:
# Mean Absolute Error (MAE)
print("MAE:", metrics.mean_absolute_error(y_test, y_predicted))

# Mean Squared Error (MSE) 
print("MSE:", metrics.mean_squared_error(y_test, y_predicted))

# Root Mean Squared Error (RMSE)
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))

# R-squared
print("R² score:", r2_score(y_test, y_predicted))

In [ ]:
# Measure how well the model explains the variation in the data (1 = perfect prediction)
eV = round(sm.explained_variance_score(y_test, y_predicted), 2)
print('Explained variance score ',eV )

In [ ]:
# Visualize the regression results
plt.title('Multiple Linear Regression')
plt.scatter(y_test, y_predicted, color='green', label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Perfect Prediction')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

The green dots represent individual countries, where the x-axis shows the actual COVID-19 death rates and the y-axis shows the predicted rates from the model. The red dashed line indicates perfect predictions. Since many green dots are far from the line, especially at higher values, it shows that the model struggles to accurately predict the death rates based on the health variables.

##### 5.2.1.1 Conclusion of multiple linear regression

<!-- Based on the results, the linear regression model dos not perform well.
The average error (MAE) is 774 and the root mean square error (RMSE) is over 1000, which means the predictions are far from the actual values.
The R² score is only 0.28, meaning that HDI explains just 28% of the differences in death rates between countries.
This suggests that HDI alone is not a good predictor of COVID-19 mortality, and that other factors likely play a more important role. -->

The multiple linear regression model gave an R² score of 0.41, meaning it explains only 41% of the variation in COVID-19 death rates. The RMSE was 929 and MAE was 781, showing that the predicted values differ quite a lot from the actual ones. This suggests that the selected health factors (like diabetes and smoking) are not strong predictors on their own. 

 #### 7.3 Conclusion of Hypothesis 4 

The results show only a weak relationship between the health conditions and COVID-19 death rates. The regression model explained just 41% of the variation (R² = 0.41), and the prediction errors (MAE = 781, RMSE = 929) were high. Only female smoking showed a moderate positive correlation.

Although research shows that chronic conditions increase the individual risk of severe illness or death from COVID-19, our hypothesis looked at country-level death rates. At the national level, many other factors — such as healthcare access, vaccination rates, and data reporting — likely affect the overall death rates. This may explain why our model only showed a weak relationship. Therefore, the hypothesis is only partially supported.

### 8. Summary and Conclusion of the Project